In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import datetime
import os
import sys
import pandas as pd
import time
sys.path.append('../..')

from src.model.two_steps.model import KNN, WeatherGenerator
from src.validation.validation import loss
import pickle
from experiments.real.config import * 
from tqdm import tqdm
from scipy.stats import wilcoxon
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import plotly.io as pio
alpha = 0.05

In [14]:
results_dir = os.path.join('..', '..', 'data', 'real', 'results_4years')

In [15]:
src_ids = [
    dir_name for dir_name in sorted(os.listdir(results_dir), reverse=True)
    if dir_name.isdigit() and os.path.exists(os.path.join(results_dir, dir_name, 'metrics_test.pickle'))
    ]
src_ids.sort()
len(src_ids)

68

In [16]:
date_start = datetime.date(2017, 1, 1)
date_end = datetime.date(2017, 12, 31)
dates = sorted(set([(date_start+datetime.timedelta(days=x)).isoformat()[5:] for x in range(0, (date_end-date_start).days+1, 7)]))
years = [str(year) for year in range(1985, 2021)]
dates = [f"{year}-{date_part}" for year in years for date_part in dates]

In [5]:
from tqdm  import tqdm
from src.model.true import check_duration
from joblib import Parallel, delayed
def parallel_observed(data, date):
    try:
        return check_duration(
            data=data, date_start=date,
            tasks_conditions=tasks_conditions, project_schedule=project_schedule)
    except Exception as e:
        print(date, e)
        return 0
results = []
results_before = []

src_durations = {}
for src_id in tqdm(src_ids):
    data = pd.read_csv(os.path.join('..', '..', 'data', 'cleaned', f"{src_id}.csv")).fillna(nan).replace([None], [nan])
    durations = (
        Parallel(n_jobs=-1, prefer="processes")
        (delayed(parallel_observed)(data=data, date=date) 
        for date in dates
        ))
    durations = {date: duration for date, duration in zip(dates, durations) if duration > 0}
    src_durations[src_id] = durations

with open(os.path.join('src_durations.pickle'), 'wb') as file:
    pickle.dump(src_durations, file, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/68 [00:11<?, ?it/s]


KeyboardInterrupt: 

In [8]:
with open(os.path.join("src_durations.pickle"), 'rb') as file:
    src_durations = pickle.load(file)

In [17]:
from scipy import stats
def quantile_before(date, duration, durations):
    distances = [abs((datetime.date.fromisoformat(_date) - datetime.date.fromisoformat(date)).days) for _date in list(durations)]
    min_distance = min(distances)
    date = next(_date for _date, distance in zip(list(durations), distances) if distance == min_distance)
    durations_before = [duration for _date, duration in durations.items() if _date[5:7] == date[5:7]]
    return int(stats.percentileofscore(durations_before, duration, kind='strict'))

In [19]:
########################################################################
# %% executando
########################################################################
results  = []
weather_pert_hpt_bests = []
for src_id in tqdm(src_ids):
    try:
        with open(os.path.join(results_dir, str(src_id), "observed.pickle"), 'rb') as file:
            observed = pickle.load(file)
        with open(os.path.join(results_dir, str(src_id), "metrics_test.pickle"), 'rb') as file:
            metrics = (
                pd.DataFrame(pickle.load(file))
                .assign(
                    duration = lambda df: df['date'].apply(lambda value: observed[value]),
                    quantile_before = lambda df: df['date'].apply(lambda value: quantile_before(date=value, duration=observed[value], durations=src_durations[src_id])))
                )#.loc[lambda df: df['date'].apply(lambda value: value[:4] != '2018')])
            p = 100*((metrics['quantile_before'].between(10, 90).sum())/len(metrics)).round(2)
            #metrics = metrics.loc[lambda df: df['quantile_before'].between(10, 90)]
        with open(os.path.join(results_dir, str(src_id), "weather_pert_hpt_best.pickle"), 'rb') as file:
            weather_pert_hpt_best = pickle.load(file)
            weather_pert_hpt_bests.append({**weather_pert_hpt_best['right-tail'], 'src_id': src_id})
        #print(src_id, len(metrics))
        #print(weather_pert_hpt_best['right-tail'])
        row = {
            'src_id': src_id
        }
        for loss in ['right-tail', 'pinball', 'density']:
            two_steps = metrics.loc[metrics['model'] == 'two_steps', loss].tolist()
            weather_pert = metrics.loc[metrics['model'] == 'weather_pert', loss].tolist()
            statistic, p_value = wilcoxon(weather_pert, two_steps)
            sig = p_value < alpha
            row = {**row, 
                f'{loss}_sig': sig,
                f'two_steps_{loss}_mean': np.mean(two_steps),
                f'two_steps_{loss}_std': np.std(two_steps),
                f'two_steps_{loss}_mean_error': np.std(two_steps) / np.sqrt(len(two_steps)),
                f'weather_pert_{loss}_mean': np.mean(weather_pert),
                f'weather_pert_{loss}_std': np.std(weather_pert),
                f'weather_pert_{loss}_mean_error': np.std(weather_pert) / np.sqrt(len(weather_pert)),
                'p': p }
        results.append(row)
    except Exception as e:
        print(e)
        continue
    

results = pd.DataFrame(results)

100%|██████████| 68/68 [00:10<00:00,  6.77it/s]


In [20]:
for loss in ['right-tail', 'pinball', 'density']:
    results[f'best_model_{loss}'] = [('two_steps' if row[f'two_steps_{loss}_mean'] < row[f'weather_pert_{loss}_mean'] else 'weather_pert')  for row in results.to_dict('records')]
    results[f'best_sig_model_{loss}'] = [(row[f'best_model_{loss}'] if row[f'{loss}_sig']  else 'not sig')  for row in results.to_dict('records')]
results.head()

,src_id,right-tail_sig,two_steps_right-tail_mean,two_steps_right-tail_std,two_steps_right-tail_mean_error,weather_pert_right-tail_mean,weather_pert_right-tail_std,weather_pert_right-tail_mean_error,p,pinball_sig,...,two_steps_density_mean_error,weather_pert_density_mean,weather_pert_density_std,weather_pert_density_mean_error,best_model_right-tail,best_sig_model_right-tail,best_model_pinball,best_sig_model_pinball,best_model_density,best_sig_model_density
0,1023,True,1.773422,0.930201,0.310067,2.233826,1.299619,0.433206,78.0,False,...,0.003464,-0.023623,0.014776,0.004925,two_steps,two_steps,two_steps,not sig,two_steps,not sig
1,1039,False,1.581765,1.990910,0.574726,1.703213,1.968012,0.568116,92.0,False,...,0.003663,-0.034388,0.013210,0.003813,two_steps,not sig,two_steps,not sig,two_steps,not sig
2,105,False,2.567969,2.706433,0.816020,2.072750,1.941326,0.585332,73.0,True,...,0.003919,-0.026334,0.010983,0.003312,weather_pert,not sig,weather_pert,weather_pert,weather_pert,not sig
3,1074,False,1.587387,0.946648,0.273274,2.086533,1.892237,0.546242,100.0,False,...,0.003577,-0.033775,0.019935,0.005755,two_steps,not sig,two_steps,not sig,weather_pert,not sig
4,1076,False,1.756733,0.961513,0.277565,1.684049,0.742992,0.214483,92.0,False,...,0.003376,-0.024085,0.010391,0.003000,weather_pert,not sig,weather_pert,not sig,two_steps,not sig


In [21]:
results['best_sig_model'] = [(
    row['best_sig_model_right-tail'] if row['best_sig_model_right-tail'] != 'not sig' else
    row['best_sig_model_pinball'] if row['best_sig_model_pinball'] != 'not sig' else
    row['best_sig_model_density']
    )  for row in results.to_dict('records')]
results['best_sig_model'].value_counts()

best_sig_model
two_steps       26
weather_pert    22
not sig         20
Name: count, dtype: int64

In [22]:
results['best_sig_model'] = [(
    row['best_sig_model_right-tail'] if row['best_sig_model_right-tail'] != 'not sig' else
    row['best_sig_model_pinball'] if row['best_sig_model_pinball'] != 'not sig' else
    row['best_sig_model_density']
    )  for row in results.to_dict('records')]
results['best_sig_model'].value_counts()

best_sig_model
two_steps       26
weather_pert    22
not sig         20
Name: count, dtype: int64

In [23]:
results[['src_id', 'best_sig_model']].merge(pd.DataFrame(weather_pert_hpt_bests), how='left', on='src_id').sort_values('best_sig_model').head(60)

,src_id,best_sig_model,alpha,smoothing,mc,cossine
35,386,not sig,1.00,14,True,True
45,613,not sig,0.95,14,True,False
19,1393,not sig,0.95,7,True,False
54,726,not sig,1.00,14,True,True
58,842,not sig,1.00,14,True,False
44,605,not sig,1.00,7,True,True
43,595,not sig,1.00,7,True,False
14,1302,not sig,1.00,7,True,True
13,1285,not sig,1.00,14,True,True
12,1215,not sig,0.95,3,True,True


In [24]:
results.head()

,src_id,right-tail_sig,two_steps_right-tail_mean,two_steps_right-tail_std,two_steps_right-tail_mean_error,weather_pert_right-tail_mean,weather_pert_right-tail_std,weather_pert_right-tail_mean_error,p,pinball_sig,...,weather_pert_density_mean,weather_pert_density_std,weather_pert_density_mean_error,best_model_right-tail,best_sig_model_right-tail,best_model_pinball,best_sig_model_pinball,best_model_density,best_sig_model_density,best_sig_model
0,1023,True,1.773422,0.930201,0.310067,2.233826,1.299619,0.433206,78.0,False,...,-0.023623,0.014776,0.004925,two_steps,two_steps,two_steps,not sig,two_steps,not sig,two_steps
1,1039,False,1.581765,1.990910,0.574726,1.703213,1.968012,0.568116,92.0,False,...,-0.034388,0.013210,0.003813,two_steps,not sig,two_steps,not sig,two_steps,not sig,not sig
2,105,False,2.567969,2.706433,0.816020,2.072750,1.941326,0.585332,73.0,True,...,-0.026334,0.010983,0.003312,weather_pert,not sig,weather_pert,weather_pert,weather_pert,not sig,weather_pert
3,1074,False,1.587387,0.946648,0.273274,2.086533,1.892237,0.546242,100.0,False,...,-0.033775,0.019935,0.005755,two_steps,not sig,two_steps,not sig,weather_pert,not sig,not sig
4,1076,False,1.756733,0.961513,0.277565,1.684049,0.742992,0.214483,92.0,False,...,-0.024085,0.010391,0.003000,weather_pert,not sig,weather_pert,not sig,two_steps,not sig,not sig


In [30]:
list(models)

['Diferença não significativa',
 'Modelo proposto foi melhor',
 'Gerador climático foi melhor']

In [38]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2, shared_xaxes=False, shared_yaxes=False,
    subplot_titles=['', '(a) Right tail pinball', '(b) Pinball', '(c) Density'],
    horizontal_spacing = 0.15,
    vertical_spacing  = 0.15)

models = {
    'Diferença não significativa': {
        'cor_ponto': 'rgba(148, 148, 148, 0.8)',
        'cor_erro': 'rgba(148, 148, 148, 0.6)',
        'value': 'not sig'
    },
    'Modelo proposto foi melhor': {
        'cor_ponto': 'rgba(254,127,12, 1)',
        'cor_erro': 'rgba(254,127,12, 0.6)',
        'value': 'weather_pert'
    },
    'Gerador climático foi melhor': {
        'cor_ponto': 'rgba(32,120,180, 1)',
        'cor_erro': 'rgba(32,120,180, 0.6)',
        'value': 'two_steps'
    },
}

for (row, col), loss in zip([(1, 2), (2, 1), (2,2)],['right-tail', 'pinball', 'density']):
    print((row, col), loss)
    range = [
        min([(results[f'two_steps_{loss}_mean'] + results[f'two_steps_{loss}_mean_error']).min(), (results[f'weather_pert_{loss}_mean'] + results[f'weather_pert_{loss}_mean_error']).min(), 0]),
        max([(results[f'two_steps_{loss}_mean'] + results[f'two_steps_{loss}_mean_error']).max(), (results[f'weather_pert_{loss}_mean'] + results[f'weather_pert_{loss}_mean_error']).max()])
    ]

    for label, config in models.items():
        _results = results.loc[lambda df: df[f'best_sig_model_{loss}'] == config['value']]
        label = {
            'Diferença não significativa': 'Non-significant difference',
            'Modelo proposto foi melhor': 'The proposed model was better',
            'Gerador climático foi melhor': 'The two-step model was better'}[label]
        fig_ = go.Scatter(
            x=_results[f'two_steps_{loss}_mean'],
            y=_results[f'weather_pert_{loss}_mean'],
            mode='markers',
            name=label,
            error_y=dict(
                type='data', # tipo de barra de erro 
                array=_results[f'weather_pert_{loss}_mean_error'], # valor do erro em y para cada ponto (acima)
                arrayminus=_results[f'weather_pert_{loss}_mean_error'], # valor do erro em y para cada ponto (abaixo)
                visible=True, color=config['cor_erro']),
            error_x=dict(
                type='data', # tipo de barra de erro 
                array=_results[f'two_steps_{loss}_mean_error'], # valor do erro em x para cada ponto (direita)
                arrayminus=_results[f'two_steps_{loss}_mean_error'], # valor do erro em x para cada ponto (esquerda)
                visible=True, color=config['cor_erro']),
            marker=dict(
                color=config['cor_ponto'], 
                size=5, line=dict(width=1, color='DarkSlateGrey')
            ),
        )
        if row != 1:
            fig_.update(showlegend=False)

        fig.add_trace(fig_, row=row, col=col)
        

        # Adicione seus traços aqui
    fig_ = go.Scatter(
        x=range,
        y=range,
        mode="lines",
        name="Identity line",
        line=dict(color='rgba(0,0,0,0.6)', dash='dash')
    )
    if row != 1:
        fig_.update(showlegend=False)
    fig.add_trace( fig_, row=row, col=col)
    fig.update_xaxes(range=range, row=row, col=col, title="Two-step model")
    fig.update_yaxes(range=range, row=row, col=col, title="Proposed model")

fig.update_layout(legend=dict(orientation='v', yanchor='bottom', y=0.8, x = 0))

fig.update_layout(
    template = "simple_white",
    width=800, height=800,
    margin=dict(t=50))


fig.write_image('exp_comparacao_modelo.pdf')#plot_url = py.plot(fig, filename='latex', include_mathjax='cdn')

(1, 2) right-tail
(2, 1) pinball
(2, 2) density


In [35]:
results.groupby(['best_sig_model_right-tail', 'best_sig_model_pinball', 'best_sig_model_density', 'best_sig_model']).agg(count = ('src_id', 'count'))

count
best_sig_model_right-tail best_sig_model_pinball best_sig_model_density best_sig_model       
not sig                   not sig                not sig                not sig            20
                                                 two_steps              two_steps           1
                                                 weather_pert           weather_pert        4
                          two_steps              not sig                two_steps           3
                                                 weather_pert           two_steps           1
                          weather_pert           not sig                weather_pert        3
                                                 weather_pert           weather_pert        2
two_steps                 not sig                not sig                two_steps           3
                                                 two_steps              two_steps           4
                          two_steps              not sig                two_steps           3
                                                 two_steps              two_steps          10
                                                 weather_pert           two_steps           1
weather_pert              not sig                not sig                weather_pert        1
                                                 two_steps              weather_pert        1
                                                 weather_pert           weather_pert        1
                          weather_pert           not sig                weather_pert        5
                                                 weather_pert           weather_pert        5

In [36]:
results

,src_id,right-tail_sig,two_steps_right-tail_mean,two_steps_right-tail_std,two_steps_right-tail_mean_error,weather_pert_right-tail_mean,weather_pert_right-tail_std,weather_pert_right-tail_mean_error,p,pinball_sig,...,weather_pert_density_mean,weather_pert_density_std,weather_pert_density_mean_error,best_model_right-tail,best_sig_model_right-tail,best_model_pinball,best_sig_model_pinball,best_model_density,best_sig_model_density,best_sig_model
0,1023,True,1.773422,0.930201,0.310067,2.233826,1.299619,0.433206,78.0,False,...,-0.023623,0.014776,0.004925,two_steps,two_steps,two_steps,not sig,two_steps,not sig,two_steps
1,1039,False,1.581765,1.990910,0.574726,1.703213,1.968012,0.568116,92.0,False,...,-0.034388,0.013210,0.003813,two_steps,not sig,two_steps,not sig,two_steps,not sig,not sig
2,105,False,2.567969,2.706433,0.816020,2.072750,1.941326,0.585332,73.0,True,...,-0.026334,0.010983,0.003312,weather_pert,not sig,weather_pert,weather_pert,weather_pert,not sig,weather_pert
3,1074,False,1.587387,0.946648,0.273274,2.086533,1.892237,0.546242,100.0,False,...,-0.033775,0.019935,0.005755,two_steps,not sig,two_steps,not sig,weather_pert,not sig,not sig
4,1076,False,1.756733,0.961513,0.277565,1.684049,0.742992,0.214483,92.0,False,...,-0.024085,0.010391,0.003000,weather_pert,not sig,weather_pert,not sig,two_steps,not sig,not sig
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,888,False,1.378250,1.100993,0.076157,1.403195,1.136204,0.078593,76.0,True,...,-0.041696,0.034007,0.002352,two_steps,not sig,two_steps,two_steps,weather_pert,not sig,two_steps
64,889,True,1.375954,1.023928,0.075281,1.447886,1.040240,0.076480,74.0,True,...,-0.035927,0.019898,0.001463,two_steps,two_steps,two_steps,two_steps,two_steps,two_steps,two_steps
65,9,True,3.852840,2.532666,0.175188,3.991644,2.609040,0.180471,74.0,True,...,-0.016744,0.013106,0.000907,two_steps,two_steps,two_steps,two_steps,two_steps,not sig,two_steps
66,908,False,1.564331,0.858000,0.059349,1.612695,1.275033,0.088196,95.0,False,...,-0.033517,0.019542,0.001352,two_steps,not sig,two_steps,not sig,weather_pert,weather_pert,weather_pert


In [39]:
results
for row in results.to_dict('records'):
    print(
        f"{row['src_id']} & "

        f"{row['weather_pert_right-tail_mean']:.2f} ± {row['weather_pert_right-tail_mean_error']:.2f} {'*' if row['best_sig_model_right-tail'] == 'weather_pert' else ''} & "
        f"{row['two_steps_right-tail_mean']:.2f} ± {row['two_steps_right-tail_mean_error']:.2f} {'*' if row['best_sig_model_right-tail'] == 'two_steps' else ''} & "

        f"{row['weather_pert_pinball_mean']:.2f} ± {row['weather_pert_pinball_mean_error']:.2f} {'*' if row['best_sig_model_pinball'] == 'weather_pert' else ''} & "
        f"{row['two_steps_pinball_mean']:.2f} ± {row['two_steps_pinball_mean_error']:.2f} {'*' if row['best_sig_model_pinball'] == 'two_steps' else ''} & "

        f"{row['weather_pert_density_mean']:.2f} ± {row['weather_pert_density_mean_error']:.2f} {'*' if row['best_sig_model_density'] == 'weather_pert' else ''} & "
        f"{row['two_steps_density_mean']:.2f} ± {row['two_steps_density_mean_error']:.2f} {'*' if row['best_sig_model_density'] == 'two_steps' else ''} & "
            
        f"{'Proposed model' if row['best_sig_model'] == 'weather_pert' else 'Two-step model' if row['best_sig_model'] == 'two_steps' else ''} \\\\ "
    )

1023 & 2.23 ± 0.43  & 1.77 ± 0.31 * & 7.43 ± 1.53  & 5.91 ± 0.93  & -0.02 ± 0.00  & -0.03 ± 0.00  & Two-step model \\ 
1039 & 1.70 ± 0.57  & 1.58 ± 0.57  & 5.16 ± 1.54  & 5.13 ± 1.54  & -0.03 ± 0.00  & -0.04 ± 0.00  &  \\ 
105 & 2.07 ± 0.59  & 2.57 ± 0.82  & 6.48 ± 1.89 * & 8.38 ± 2.24  & -0.03 ± 0.00  & -0.02 ± 0.00  & Proposed model \\ 
1074 & 2.09 ± 0.55  & 1.59 ± 0.27  & 6.20 ± 1.45  & 5.33 ± 0.97  & -0.03 ± 0.01  & -0.03 ± 0.00  &  \\ 
1076 & 1.68 ± 0.21  & 1.76 ± 0.28  & 5.45 ± 0.70  & 6.08 ± 1.15  & -0.02 ± 0.00  & -0.03 ± 0.00  &  \\ 
1090 & 0.98 ± 0.20  & 1.02 ± 0.17  & 3.05 ± 0.63  & 3.15 ± 0.57  & -0.05 ± 0.01  & -0.04 ± 0.00  &  \\ 
1145 & 1.63 ± 0.27  & 1.48 ± 0.26  & 5.51 ± 1.09  & 4.93 ± 1.03 * & -0.03 ± 0.00  & -0.03 ± 0.00  & Two-step model \\ 
1161 & 2.62 ± 0.74  & 2.42 ± 0.61  & 8.22 ± 1.85  & 7.72 ± 1.56  & -0.02 ± 0.00  & -0.02 ± 0.00  &  \\ 
1171 & 3.08 ± 0.53  & 3.36 ± 0.53  & 10.97 ± 2.01  & 11.58 ± 1.92  & -0.02 ± 0.00  & -0.02 ± 0.00  &  \\ 
1180 & 1.52 ± 0.35

In [23]:
results

,src_id,right-tail_sig,two_steps_right-tail_mean,two_steps_right-tail_std,two_steps_right-tail_mean_error,weather_pert_right-tail_mean,weather_pert_right-tail_std,weather_pert_right-tail_mean_error,pinball_sig,two_steps_pinball_mean,...,weather_pert_density_mean,weather_pert_density_std,weather_pert_density_mean_error,best_model_right-tail,best_sig_model_right-tail,best_model_pinball,best_sig_model_pinball,best_model_density,best_sig_model_density,best_sig_model
0,1033,False,1.612476,0.981766,0.111163,1.480321,0.905745,0.102555,True,5.822259,...,-0.028219,0.011750,0.001330,weather_pert,not sig,weather_pert,weather_pert,weather_pert,not sig,weather_pert
1,105,False,1.625576,1.098313,0.122795,1.475766,0.818271,0.091485,True,6.090128,...,-0.029395,0.008798,0.000984,weather_pert,not sig,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
2,1055,True,2.364595,1.817675,0.177387,1.903278,1.142214,0.111469,True,7.671965,...,-0.023657,0.011089,0.001082,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
3,113,True,1.589608,0.936909,0.091433,1.313698,0.910599,0.088865,True,5.060084,...,-0.038801,0.020881,0.002038,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
4,1137,True,1.995837,1.994950,0.194687,1.740887,1.807490,0.176393,True,6.125418,...,-0.035626,0.021501,0.002098,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,888,True,1.642928,1.503227,0.146700,2.451463,2.146676,0.209494,True,5.110451,...,-0.033547,0.036817,0.003593,two_steps,two_steps,two_steps,two_steps,two_steps,two_steps,two_steps
61,889,True,1.261704,1.415050,0.138095,1.895055,2.126478,0.207523,True,4.174145,...,-0.039878,0.032583,0.003180,two_steps,two_steps,two_steps,two_steps,two_steps,not sig,two_steps
62,9,True,4.829371,2.644249,0.258052,5.019689,2.715453,0.265001,True,16.722446,...,-0.011979,0.011943,0.001165,two_steps,two_steps,two_steps,two_steps,two_steps,not sig,two_steps
63,908,True,1.592974,0.936059,0.091350,1.608262,0.926271,0.090395,False,5.062485,...,-0.028205,0.011095,0.001083,two_steps,two_steps,two_steps,not sig,two_steps,not sig,two_steps
